## Online CNMF-E

This demo shows an example of doing online analysis on one-photon data. We compare offline and online approaches. The dataset used is courtesy of the Miniscope project.

In [ ]:
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')

from IPython.display import display, clear_output
import glob
import logging
import numpy as np
import os
import scipy

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)10s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.WARNING)

import caiman as cm
from caiman.source_extraction import cnmf as cnmf
from caiman.motion_correction import MotionCorrect
from caiman.utils.utils import download_demo
import matplotlib.pyplot as plt
from caiman.utils.visualization import nb_inspect_correlation_pnr
import holoviews as hv
import bokeh.plotting as bpl

bpl.output_notebook()
hv.notebook_extension('bokeh')

### Select file(s) to be processed
The `download_demo` function will download the specific file for you and return the complete path to the file which will be stored in your `caiman_data` directory. If you adapt this demo for your data make sure to pass the complete path to your file(s). Remember to pass the `fnames` variable as a list. Note that the memory requirement of the offline CNMF-E algorithm are much higher compared to the standard CNMF algorithm. One of the benefits of the online approach is the reduced memory requirements.

In [ ]:
fnames = [download_demo('msCam13.avi')]

## Batch (offline) approach

We start with motion correction and then proceed with the source extraction using the CNMF-E algorithm. For a detailed 1p demo check `demo_pipeline_cnmfE.ipynb`.

In [ ]:
# motion correction parameters
motion_correct = True            # flag for performing motion correction
pw_rigid = False                 # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (7, 7)               # size of high pass spatial filtering, used in 1p data
max_shifts = (15, 15)            # maximum allowed rigid shift
strides = (96, 96)               # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)              # overlap between pathes (size of patch strides+overlaps)
border_nan = 'copy'              # replicate values along the boundaries

mc_dict = {
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'border_nan': border_nan
}

opts = cnmf.params.CNMFParams(params_dict=mc_dict)

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

In [ ]:
mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
mc.motion_correct(save_movie=True)

### inspect motion correction results

In [ ]:
inspect_results = True
if inspect_results:
    cm.concatenate((cm.load(fnames), cm.load(mc.mmap_file)), axis=1).play()
plt.figure(); plt.plot(mc.shifts_rig); plt.legend(['x-shifts', 'y-shifts'])    

The motion correction results look good. We then proceed with memory mapping and checking the correlation/pnr images.

In [ ]:
from time import time
fname_new = cm.save_memmap(mc.mmap_file, base_name='memmap_', order='C',
                           border_to_0=0, dview=dview)
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')

### Inspect correlation and PNR images to set relevant thresholds

In [ ]:
gSig = (5, 5)

cn_filter, pnr = cm.summary_images.correlation_pnr(images[::max(T//1000, 1)], gSig=gSig[0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
# inspect the summary images and set the parameters

nb_inspect_correlation_pnr(cn_filter, pnr)

## Set parameters for source extraction
From the images above we select `min_pnr = 6` and `min_corr = 0.8`. We pass these alongside the other parameters needed for offline 1p processing.

In [ ]:
min_pnr = 6
min_corr = 0.8
rf = 48                                        # half size of each patch
stride = 8                                     # amount of overlap between patches     
ssub = 1                                       # spatial downsampling factor   
decay_time = 0.4                               # length of typical transient (in seconds)
fr = 10                                        # imaging rate (Hz) 
gSig = (5, 5)                                  # expected half size of neurons
gSiz = (15, 15)                                # half size for neuron bounding box   
p = 0                                          # order of AR indicator dynamics
min_SNR = 1.5                                  # minimum SNR for accepting new components
rval_thr = 0.85                                # correlation threshold for new component inclusion
merge_thr = 0.65                               # merging threshold
K = None                                       # initial number of components

cnmfe_dict = {'fnames': fnames,
              'fr': fr,
              'decay_time': decay_time,
              'method_init': 'corr_pnr',
              'gSig': gSig,
              'gSiz': gSiz,
              'rf': rf,
              'stride': stride,
              'p': p,
              'nb': 0,
              'ssub': ssub,
              'min_SNR': min_SNR,
              'min_pnr': min_pnr,
              'min_corr': min_corr,
              'bas_nonneg': False,
              'center_psf': True,
              'rval_thr': rval_thr,
              'only_init': True,
              'merge_thr': merge_thr,
              'K': K}
opts.change_params(cnmfe_dict);

In [ ]:
from time import time
t1 = -time()
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, params=opts)
cnm.fit(images)
t1 += time()

## View the results

In [ ]:
cnm.estimates.plot_contours_nb(img=pnr)

In [ ]:
cnm.estimates.hv_view_components(img=cn_filter)

### Show a movie with the results

In [ ]:
cnm.estimates.play_movie(images, magnification=0.75, include_bck=False)

## Online Processing
Now try the online approach. The idea behind the online algorithm is simple:
- First initialize the estimates by running the batch (offline) algorithm in small subset.
- Then process each frame as it arrives. The processing consists of:
    * Motion correct the new frame
    * Extract the activity of existing neurons at this frame, and neuropil
    * Search for new neurons that appear in this frame and have not been detected earlier.
- Periodically update shapes of existing neurons and background model.

## Setup additional parameters for online processing

In [ ]:
from copy import deepcopy
online_opts = deepcopy(cnm.params)

In [ ]:
rf = 48
stride = 8
ssub = 1
ds_factor = 2*ssub

gSig = (6//ds_factor, 6//ds_factor)            # expected half size of neurons
gSiz = (15//ds_factor, 15//ds_factor)
sniper_mode = False                            # flag using a CNN to detect new neurons (o/w space correlation is used)
init_batch = 300                               # number of frames for initialization (presumably from the first file)
expected_comps = 500                           # maximum number of expected components used for memory pre-allocation (exaggerate here)
dist_shape_update = False                      # flag for updating shapes in a distributed way
min_num_trial = 5                              # number of candidate components per frame     
K = None                                       # initial number of components
epochs = 1                                     # number of passes over the data
show_movie = False                             # show the movie with the results as the data gets processed

online_dict = {'epochs': epochs,
               'nb': 0,
               'ssub': ssub,
               'ds_factor': ds_factor,                                   # ds_factor >= ssub should hold
               'gSig': gSig,
               'gSiz': gSiz,
               'gSig_filt': (3, 3),
               'min_corr': min_corr,
               'bas_nonneg': False,
               'center_psf': True,
               'max_shifts_online': 15,
               'rval_thr': rval_thr,
               'init_batch': init_batch,
               'only_init': True,
               'init_method': 'cnmf',
               'normalize_init': False,
               'update_freq': 200,
               'expected_comps': expected_comps,
               'sniper_mode': sniper_mode,
               'dist_shape_update' : dist_shape_update,
               'min_num_trial': min_num_trial,
               'epochs': epochs,
               'show_movie': show_movie}
online_opts.change_params(online_dict);

In [ ]:
cnm_online = cnmf.online_cnmf.OnACID(params=online_opts, dview=dview)
cnm_online.fit_online()

In [ ]:
#images = cm.load(fnames[0], subindices=slice(0,1000))
#Cn, pnr = cm.summary_images.correlation_pnr(images[::1], gSig=gSig[0], swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
cnm_online.estimates.nb_view_components(img=pnr, denoised_color='red');

In [ ]:
cnm_online.estimates.plot_contours_nb(img=pnr)

## Plot timing
The plot below shows the time spent on each part of the algorithm (motion correction, tracking of current components, detect new components, update shapes) for each frame. Note that if you displayed a movie while processing the data (`show_movie=True`) the time required to generate this movie will be included here.

In [ ]:
T_motion = 1e3*np.array(cnm_online.t_motion)
T_detect = 1e3*np.array(cnm_online.t_detect)
T_shapes = 1e3*np.array(cnm_online.t_shapes)
T_online = 1e3*np.array(cnm_online.t_online) - T_motion - T_detect - T_shapes
plt.figure()
plt.stackplot(np.arange(len(T_motion)), T_motion, T_online, T_detect, T_shapes)
plt.legend(labels=['motion', 'process', 'detect', 'shapes'], loc=2)
plt.title('Processing time allocation')
plt.xlabel('Frame #')
plt.ylabel('Processing time [ms]')
plt.ylim([0, 1.2e3*np.percentile(np.array(cnm_online.t_online), 90)]);